# AI Jukebox

### Generate Midi File
Plug in trained weights and notes file to generate midi.  

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import sys
from datetime import datetime
import re
import pickle
# from music21 import instrument

import processing as pr
import neural_network as nn
import generate as cr
import utils as ut

/home/cipher000/anaconda3/envs/MusicGenerator1/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/cipher000/anaconda3/envs/MusicGenerator1/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
weight_file = 'output/201803281645-Tadpole-1-100-weight-01-3.1328-2.3204.hdf5'
note_file = 'output/201803281647-Tadpole-10-100-input_notes'

timestamp = re.sub(r'[-: ]','',str(datetime.now()).split('.')[0])[:-2]
# output_name = midi_files.split('/')[-3]
# epochs = 200
# batch_size = 128
sequence_length = 100 # the LSTM RNN will consider this many notes
output_tag = 'output/{}-{}-'.format(timestamp, sequence_length)
notes_generated = 500

In [3]:
# convert fully trained weights to midi file
def weights_to_midi(note_file, sequence_length, weight_file):
    with open(note_file, 'rb') as filepath:
        notes = pickle.load(filepath)
    network_input, network_output, n_patterns, n_vocab, pitchnames = pr.prepare_sequences(notes, sequence_length)
    normalized_input = pr.reshape_for_creation(network_input, n_patterns, sequence_length, n_vocab)
    model = nn.create_network(normalized_input, n_vocab, weight_file)
    prediction_output= cr.generate_notes(model, network_input, pitchnames,notes_generated, n_vocab)
    output_notes, midi_file = cr.create_midi(prediction_output, output_tag, sequence_length)
    return output_notes, midi_file

In [4]:
# terminal_output = output_tag + 'terminal.log'
# sys.stdout = ut.Logger(terminal_output)
# print("Terminal output being saved at {}".format(terminal_output))
output_notes, midi_file = weights_to_midi(note_file, sequence_length, weight_file)
# print("Run Complete. Terminal log saved at {}".format(terminal_output))


**Preparing sequences for training**
Pitchnames (unique notes/chords from 'notes') at length 78: ['0.3.7', '1.4', '1.6', '1.7', '10.2.5', '11.2.4', '11.2.6', '11.4', '2.6', '2.6.9', '2.7', '3.7.10', '4.7', '4.7.11', '4.7.9', '4.9', '5.9.0', '6.9', '7.10.2', '7.11', '7.11.2', '9', '9.1.4', '9.2', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'B-2', 'B-3', 'B-4', 'B-5', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'C#3', 'C#4', 'C#5', 'C#6', 'C2', 'C3', 'C4', 'C5', 'C6', 'D2', 'D3', 'D4', 'D5', 'D6', 'E-3', 'E-4', 'E-5', 'E0', 'E2', 'E3', 'E4', 'E5', 'E6', 'F#3', 'F#4', 'F#5', 'F#6', 'F2', 'F4', 'F5', 'G#4', 'G#5', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6']
Note to integer embedding created at length 78
Network input and output created with (pre-transform) lengths 159689 and 159689
Network input and output first list items: [75, 27, 37, 75, 60, 75, 51, 75, 60, 75, 47, 27, 27, 27, 37, 47, 27, 27, 37, 75, 60, 75, 51, 75, 60, 75, 47, 37, 27, 37, 75, 75, 27, 37, 75, 60, 75, 51, 75, 60, 75, 47, 27, 27, 27, 37, 47, 27

ValueError: Dimension 1 in both shapes must be equal, but are 78 and 7 for 'Assign_12' (op: 'Assign') with input shapes: [1024,78], [1024,7].

### Analyze created MIDI: Music21

In [ ]:
import pretty_midi
import matplotlib.pyplot as plt
import librosa.display
import pandas as pd
import IPython.display as ipd

In [ ]:
fs = 16000
rate = 16000
start_pitch = 24
end_pitch = 84

In [ ]:
def plot_piano_roll(pm, start_pitch, end_pitch, fs=10):
    librosa.display.specshow(pm.get_piano_roll(fs)[start_pitch:end_pitch],
                             hop_length=1, sr=fs, x_axis='time', y_axis='cqt_note',
                             fmin=pretty_midi.note_number_to_hz(start_pitch))
    
def analyze_midi(midi_file):
    instrument_dict = {}
    pm = pretty_midi.PrettyMIDI(midi_file)
    plt.figure(figsize=(12, 6))
    plot_piano_roll(pm, start_pitch, end_pitch)
    print("Time Signature Changes: {} Instruments: {}".format(len(pm.time_signature_changes), len(pm.instruments)))
    for instrument_num in range(len(pm.instruments)):
        instrument_dict[instrument_num] = {'Notes': len(pm.instruments[instrument_num].notes), 
                                           'Pitch_Bends':len(pm.instruments[instrument_num].pitch_bends),
                                           'Control_Changes':len(pm.instruments[instrument_num].control_changes)}
    instrument_dict = pd.DataFrame(instrument_dict)
    return instrument_dict, pm

In [ ]:
# midi_file = 'output/201803281154-100-lstm_midi.mid'
instrument_dict,pm = analyze_midi(midi_file)
ipd.Audio(pm.synthesize(fs=fs), rate=rate)

In [ ]:
output_notes

In [ ]:
from music21 import *
us = environment.UserSettings()
us.create()

In [ ]:
for key in sorted(us.keys()):
    print(key)